In [ ]:
!git clone https://github.com/lanzhang128/multi_agent_autoformalization.git
%cd multi_agent_autoformalization
!mkdir test_results

Cloning into 'multi_agent_autoformalization'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 65 (delta 29), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 24.63 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/multi_agent_autoformalization


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xzvf /content/drive/MyDrive/Isabelle2024_linux.tar.gz -C /content/

流式输出内容被截断，只能显示最后 5000 行内容。
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test102.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test036.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test041.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test018.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test035.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test007.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test103.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test149.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test167.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test118.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test042.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test069.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test099.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test173.ML
Isabelle2024/contrib/polyml-5.9.1/src/Tests/Succeed/Test065.ML
Isabelle2024/contrib/polyml-

In [ ]:
import os
os.environ['ISABELLE_DIRPATH'] = os.path.abspath('/content/Isabelle2024')

import nest_asyncio
nest_asyncio.apply()

#Settings

### Instantiate LLMs

In [ ]:
from llm import OpenAILLM

api_key = 'api-key'
gpt_4o = OpenAILLM(
    name='gpt-4o',
    api_key=api_key,
    model='gpt-4o')

In [ ]:
from llm import HuggingFaceLLM

deepseekmath = HuggingFaceLLM(
    name='deepseekmath',
    model_id='deepseek-ai/deepseek-math-7b-instruct')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model-00001-of-00002.bin:  76%|#######5  | 7.54G/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

###Set Formal Language

In [ ]:
formal_language = 'Isabelle/HOL'

###Instantiate an autoformalization agent

In [ ]:
from agent import AutoformalizationAgent

agent_autoformalization = AutoformalizationAgent(
    llm=gpt_4o,
    name='autoformalization',
    formal_language=formal_language)

###Instantiate a denoising agent

In [ ]:
from agent import DenoisingAgent

agent_denoising = DenoisingAgent(
    llm=gpt_4o,
    name='denoising',
    formal_language=formal_language)

###Instantiate a hard critique agent

In [ ]:
from agent import HardCritiqueAgent

agent_hard = HardCritiqueAgent(
    name='theorem prover',
    formal_language=formal_language,
    file_dir='test_results')

logging file is at: test_results/isabelle.log
Isabelle server started with info: server "test" = 127.0.0.1:41899 (password "bea81834-aee0-482e-864d-e95b5a3534b4")
 in 2.36s.
Isabelle server session 0f46c733-74ee-4fd6-9bdf-b0fdd3d6d5e6 started in 13.09s.


###Instantiate a formal refinement agent

In [ ]:
from agent import FormalRefinementAgent

agent_formal = FormalRefinementAgent(
    llm=gpt_4o,
    name='formal refinement',
    formal_language=formal_language,
    category='none')

###Instantiate a soft critique agent

In [ ]:
from agent import SoftCritiqueAgent

aspect_description='whether the formalized code involves all mathematical concepts in the natural language statement.'

agent_soft = SoftCritiqueAgent(
    llm=gpt_4o,
    name='soft critique',
    formal_language=formal_language,
    aspect_description=aspect_description)

###Instantiate an informal refinement agent

In [ ]:
from agent import InformalRefinementAgent

agent_informal = InformalRefinementAgent(
    llm=gpt_4o,
    name='informal refinement',
    formal_language=formal_language)

###Prepare three examples for few-shot autoformalization

In [ ]:
informal_formal_pairs = [
    [
        "Determine the value of $ab$ if $\\log_8a+\\log_4b^2=5$ and $\\log_8b+\\log_4a^2=7$. Show that it is 512.",
        "theory aime_1984_p5 imports Complex_Main\nbegin\n\ntheorem aime_1984_p5:\n  fixes a b ::real\n  assumes "
        "\"(ln a) / (ln 8) + (ln (b^2)) / (ln 4) = 5\"\n          \"(ln b) / (ln 8) + (ln (a^2)) / (ln 4) = 7\"\n "
        "       shows \"a * b = 512\"\n  sorry\n\nend\n"
        ],
    [
        "One dimension of a cube is increased by $1$, another is decreased by $1$, and the third is left "
        "unchanged. The volume of the new rectangular solid is $5$ less than that of the cube. What was the "
        "volume of the cube?\n\n$\\textbf{(A)}\\ 8 \\qquad \\textbf{(B)}\\ 27 \\qquad \\textbf{(C)}\\ 64 \\qquad "
        "\\textbf{(D)}\\ 125 \\qquad \\textbf{(E)}\\ 216$ Show that it is \\text{(D)}.",
        "theory amc12a_2009_p5 imports\n  Complex_Main\nbegin\n\ntheorem amc12a_2009_p5:\n  fixes x :: real\n  "
        "assumes h0 : \"x^3 - (x+1) * (x-1) * x = 5\"\n  shows \"x^3 = 125\"\n  sorry\n\nend"
        ],
    [
        "Solve the system of equations\n\n$|a_1 - a_2| x_2 +|a_1 - a_3| x_3 +|a_1 - a_4| x_4 = 1\\\\ |a_2 - a_1| "
        "x_1 +|a_2 - a_3| x_3 +|a_2 - a_4| x_4 = 1\\\\ |a_3 - a_1| x_1 +|a_3 - a_2| x_2 +|a_3-a_4|x_4= 1\\\\ |a_4 "
        "- a_1| x_1 +|a_4 - a_2| x_2 +|a_4 - a_3| x_3 = 1$\n\nwhere $a_1, a_2, a_3, a_4$ are four different real "
        "numbers.",
        "theory imo_1966_p5 imports\nComplex_Main\n\nbegin\n\ntheorem imo_1966_p5:\n  fixes x a :: \"nat "
        "\\<Rightarrow> real\"\n  assumes \"a 1 > a 2\" and \"a 2 > a 3\" and \"a 3 > a 4\"\n  assumes \n    h6 : "
        "\"abs (a 1 - a 2) * x 2 + abs (a 1 - a 3) * x 3 + abs (a 1 - a 4) * x 4 = 1\"\n    and h7 : \"abs (a 2 - "
        "a 1) * x 1 + abs (a 2 - a 3) * x 3 + abs (a 2 - a 4) * x 4 = 1\"\n    and h8 : \"abs (a 3 - a 1) * x 1 + "
        "abs (a 3 - a 2) * x 2 + abs (a 3 - a 4) * x 4 = 1\"\n    and h9 : \"abs (a 4 - a 1) * x 1 + abs (a 4 - a "
        "2) * x 2 + abs (a 4 - a 3) * x 3 = 1\"\n  shows \"x 2 = 0 \\<and> x 3 = 0 \\<and> x 1 = 1 / abs (a 1 - a "
        "4) \\<and> x 4 = 1 / abs (a 1 - a 4)\"\n  sorry\n\nend"
        ]
    ]

###Prepare a test natural language mathematical statement

In [ ]:
informal = (
    "Find the value of $a_2+a_4+a_6+a_8+\\ldots+a_{98}$ if $a_1$, $a_2$, $a_3\\ldots$ is an [[arithmetic progression]] "
    "with common difference 1, and $a_1+a_2+a_3+\\ldots+a_{98}=137$. Show that it is 093."
)

In [ ]:
informal = (
    "Definition of Softmax Function: Formally, the standard (unit) softmax function "
    "\\(\\sigma\\colon \\mathbb{R}^K \\to (0, 1)^K\\), where \\(K \\ge 1\\), takes a vector "
    "\\(\\mathbf{z} = (z_1, \\dotsc, z_K) \\in \\mathbb{R}^K\\) and computes each component of vector "
    "\\(\\sigma(\\mathbf{z}) \\in (0, 1)^K\\) with \\(\\sigma(\\mathbf{z})_i = \\frac{e^{z_i}}{\\sum_{j=1}^K e^{z_j}}\\,.\\)"
)

In [ ]:
informal = (
    "Definition of Isotropic Position: A probability distribution over vectors is said to be in isotropic"
    "position if its covariance matrix is equal to the identity matrix. Formally, let \\(D\\) be a "
    "distribution over vectors in the vector space \\(\\mathbb{R}^n\\). Then \\(D\\) is in isotropic "
    "position if, for vector \\(v\\) sampled from the distribution, \\(\\mathbb{E}\\, vv^\\mathsf{T} = "
    "\\mathrm{Id}.\\)"
    )

# Test Agents

GPT-4o zero-shot

In [ ]:
zero_formalization, response = agent_autoformalization(informal_statement=informal)
print('formalization:')
print(zero_formalization)
print('response:')
print(response)

formalization:
theory Arithmetic_Progression
  imports Main
begin

  definition a :: "nat ⇒ nat" where "a n = n"

  lemma sum_arithmetic_progression:
    "∑ (i::nat) {1..98} a i = 137"
    by (simp add: a_def)

  lemma sum_even_terms:
    "∑ (i::nat) {2..98} a i = 93"
    by (simp add: a_def)

end

response:
%%%%%%%%%%
theory Arithmetic_Progression
  imports Main
begin

  definition a :: "nat ⇒ nat" where "a n = n"

  lemma sum_arithmetic_progression:
    "∑ (i::nat) {1..98} a i = 137"
    by (simp add: a_def)

  lemma sum_even_terms:
    "∑ (i::nat) {2..98} a i = 93"
    by (simp add: a_def)

end
%%%%%%%%%%


GPT-4o 3-shot

In [ ]:
shot_formalization, _ = agent_autoformalization(
    informal_statement=informal,
    informal_formal_pairs=informal_formal_pairs)
print('formalization:')
print(shot_formalization)

formalization:
theory isotropic_position imports
  Complex_Main
begin

definition isotropic_position :: "(real^n) set ⇒ bool" where
  "isotropic_position D ⟷ (∃v. v ∈ D ⟹ (∫ x∈D. x * x\<^sup>T) = Id)"

end



GPT-4o denoising

In [ ]:
cleaned_formalization, _ = agent_denoising(formalization=shot_formalization)
print('formalization:')
print(cleaned_formalization)

formalization:
theory arithmetic_progression imports Complex_Main
begin

theorem arithmetic_progression_sum:
  fixes a1 :: real
  assumes "a1 + (a1 + 1) + (a1 + 2) + ... + (a1 + 97) = 137"
  shows "a1 + (a1 + 2) + (a1 + 4) + ... + (a1 + 96) = 93"

end

response:
%%%%%%%%%%
theory Arithmetic_Progression
  imports Main
begin

  definition a :: "nat ⇒ nat" where "a n = n"

  lemma sum_arithmetic_progression:
    "∑ (i::nat) {1..98} a i = 137"
    by (simp add: a_def)

  lemma sum_even_terms:
    "∑ (i::nat) {2..98} a i = 93"
    by (simp add: a_def)

end
%%%%%%%%%%


Hard critique of GPT-4o 3-shot formalization

In [ ]:
correctness, error_details = agent_hard(
    formalization=shot_formalization,
    file_prefix='test')
print(correctness)
print(error_details)

Isabelle server use_theory ended in 2.26s.
False
Identified error on line: 6
Error message: Inner syntax error\<^here>
Failed to parse type




Formal refinement of GPT-4o 3-shot formalization

In [ ]:
detailed_refinement, response = agent_formal(
    informal_statement=informal,
    refinement_mode='detailed',
    formalization=shot_formalization,
    correctness=correctness,
    error_details=error_details)
print('formalization:')
print(detailed_refinement)

formalization:
theory isotropic_position imports
  Complex_Main
begin

definition isotropic_position :: "(real^n) set ⇒ bool" where
  "isotropic_position D ⟷ (∫ x∈D. x * x\<^sup>T) = Id"

end



Soft critique of GPT-4o 3-shot formalization

In [ ]:
aspect_evaluation, _ = agent_soft(
    informal_statement=informal,
    formalization=shot_formalization)
print('aspect_evaluation of :', aspect_description)
print(aspect_evaluation)

aspect_evaluation of : whether the formalized code involves all mathematical concepts in the natural language statement.
Judgement: False
Explanation: The formalization does not correctly capture the definition of isotropic position as stated in the natural language description. The definition requires that the covariance matrix, which is represented by the integral \(\mathbb{E}[vv^\mathsf{T}]\), equals the identity matrix. However, the formal code uses an existential quantifier \(\exists v\) which implies that there exists some vector \(v\) in the distribution \(D\) such that the integral condition holds, rather than stating that the integral itself must equal the identity matrix for all vectors sampled from \(D\). Additionally, the notation used for the integral is not standard; it should explicitly denote the expectation over the distribution \(D\). Therefore, the formalization does not meet the requirement of accurately representing the definition of isotropic position. 



Informal refinement of GPT-4o 3-shot formalization

In [ ]:
informal_refinement, _ = agent_informal(
    informal_statement=informal,
    formalization=shot_formalization,
    aspect_description=aspect_description,
    aspect_evaluation=aspect_evaluation)
print('formalization:')
print(detailed_refinement)

formalization:
theory isotropic_position imports
  Complex_Main
begin

definition isotropic_position :: "(real^n) set ⇒ bool" where
  "isotropic_position D ⟷ (∫ x∈D. x * x\<^sup>T) = Id"

end



Terminate the backend theorem prover

In [ ]:
agent_hard.theorem_prover.terminate()

Isabelle server 127.0.0.1:39151 shutdown.
